In [1]:
!pip3 install -U pip
!pip3 install -U setuptools wheel

# CPU version of pytorch has smaller footprint - see installation instructions in
# pytorch documentation - https://pytorch.org/get-started/locally/
!pip3 install torch==1.10.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

!pip3 install autogluon

     |████████████████████████████████| 2.1 MB 4.4 MB/s            
  Attempting uninstall: pip
    Found existing installation: pip 21.3.1
    Uninstalling pip-21.3.1:
      Successfully uninstalled pip-21.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.0
    Uninstalling wheel-0.37.0:
      Successfully uninstalled wheel-0.37.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.5.0
    Uninstalling setuptools-59.5.0:
      Successfully uninstalled setuptools-59.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
beatrix-jupyterlab 3.1.6 requires google-clou

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon

In [3]:
train_data = TabularDataset('../input/petfinder-train-test-split/train.csv')

Loaded data from: ../input/petfinder-train-test-split/train.csv | Columns = 14 / 14 | Rows = 7929 -> 7929


In [4]:
train_data.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,ec07599018885a44734a9ba27d306055,0,1,1,1,0,0,0,0,0,0,0,0,99
1,df5a9f783ff2ae373ee99fa33679da64,0,1,1,1,0,0,1,0,0,0,0,0,67
2,d9ba322c3fbf962b458c3be9a2f0ffd0,0,1,1,0,0,0,0,0,0,0,0,0,58
3,c9bd7f7c498a5189bae9eab446cd8e01,0,1,1,1,0,0,0,1,0,0,1,0,49
4,700db4fae0b34f8648ce3d96852232dc,0,1,1,1,0,0,0,0,0,0,0,0,35


In [5]:
label = 'Pawpularity'

In [6]:
save_path = 'petfinder-autogluon'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "petfinder-autogluon/"
AutoGluon Version:  0.4.0
Python Version:     3.7.12
Operating System:   Linux
Train Data Rows:    7929
Train Data Columns: 13
Label Column: Pawpularity
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 100) unique label values:  [99, 67, 58, 49, 35, 22, 23, 32, 36, 51]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.9945768697187539
Train Data Class Count: 93
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    17906.9 MB
	Train Data (Original)  Memory U

	0.0342	 = Validation score   (accuracy)
	22.98s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM ...
	0.0342	 = Validation score   (accuracy)
	22.26s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.0317	 = Validation score   (accuracy)
	1.22s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.0317	 = Validation score   (accuracy)
	1.22s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: CatBoost ...
	0.0368	 = Validation score   (accuracy)
	29.22s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.0317	 = Validation score   (accuracy)
	1.13s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.0317	 = Validation score   (accuracy)
	1.22s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: XGBoost ...
	0.033	 = Validation score   (accuracy)
	22.61s	 = Training   runtime
	0.01s	 = Valid

In [7]:
test_data_nolab = TabularDataset('../input/petfinder-train-test-split/test.csv')

Loaded data from: ../input/petfinder-train-test-split/test.csv | Columns = 13 / 13 | Rows = 1983 -> 1983


In [8]:
y_pred = predictor.predict(test_data_nolab)

In [9]:
y_pred

0       30
1       33
2       30
3       26
4       29
        ..
1978    28
1979    30
1980    26
1981    30
1982    30
Name: Pawpularity, Length: 1983, dtype: int64

In [10]:
id_ = ['Id']

In [11]:
sub = test_data_nolab[id_].copy()
sub[label] = list(y_pred.values)

In [12]:
sub.to_csv("./autogluon_sub_petfinder.csv", index = False)

In [13]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

solution = pd.read_csv('../input/petfinder-train-test-split/solution.csv')

In [14]:
solution.head()

,Id,Pawpularity
0,48d53aeabee4f92f77eee3a323343c77,40
1,b60e82fd313066b801fa4431d1ce4f4e,64
2,09ae71fc4eda1e0ae05680d1950bc009,26
3,263879abce68de4af02ef5f7ef873d24,73
4,4ea0587a137c7983ca92851b6cb36ca1,39


In [15]:
sub.head()

,Id,Pawpularity
0,48d53aeabee4f92f77eee3a323343c77,30
1,b60e82fd313066b801fa4431d1ce4f4e,33
2,09ae71fc4eda1e0ae05680d1950bc009,30
3,263879abce68de4af02ef5f7ef873d24,26
4,4ea0587a137c7983ca92851b6cb36ca1,29


In [16]:
all(solution['Id'] == sub['Id'])

True

In [17]:
from sklearn.metrics import mean_squared_error
y_true = solution['Pawpularity']
y_pred = sub['Pawpularity']
rmse = mean_squared_error(y_true, y_pred, squared = False)
rmse

23.173293094838808